# Ceneo Scraper

## Imports

In [57]:
import os
import json
import requests
from bs4 import  BeautifulSoup

## Definition of exctraction function

In [58]:
def extract_content(ancestor,selector=None, attribute=None,return_list=False):
    if selector:

        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:

                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
        return ancestor.select_one(selector).text.strip()
    if attribute:

        return ancestor[attribute]
    return ancestor.text.strip()


##  dictionary with Opinion structure

In [64]:

selectors= { 
    "opinion_id": (None,'data-entry-id'), 
    "author": ("span.user-post__author-name",), 
    "authors_recommendation": ("span.user-post__author-recomendation > em",), 
    "stars": ("span.user-post__score-count",), 
    "content": ("div.user-post__text",), 
    "pros": ("div.review-feature__title--positives ~ div.review-feature__item",None , True), 
    "cons": ("div.review-feature__title--negatives ~ div.review-feature__item", None, True), 
    "helpful": ("button.vote-yes > span",), 
    "unhelpful": ("button.vote-no > span",), 
    "publish_date": ("span.user-post__published > time:nth-child(1)","datetime"), 
    "purchase_date": ("span.user-post__published > time:nth-child(2)","datetime"), 
}

# Send request to Ceneo.pl service

In [68]:
product_id = "108290707"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

## Convert plain text HTML code to DOM structure

##  components of single opinion

|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|span.user-post__author-name|author|
|author’s recommendation|span.user-post__author-recomendation em|recommendation|
|score expressed in number of stars|span.user-post__score-count|score|
|opinion’s content|div.user-post__text|content|
|list of product advantages|div.review-feature--positives ~ div.review-feature__item|advantages|
|list of product disadvantages|div.review-feature--negatives ~ div.review-feature__item |disadvantages|
|how many users think that opinion was helpful|button.vote-yes > span|helpful|
|how many users think that opinion was unhelpful|button.vote-no > span|not_helpful|
|publishing date|span.user-post__published > time:nth-child(1)["datetime"]|pub_date|
|purchase date|span.user-post__published > time:nth-child(2)["datetime"]|pur_date|



## extract all opinions about product

In [71]:
all_opinions=[] 
while(url): 
    print(url)
    response = requests.get(url) 
    response.status_code 
    page_dom = BeautifulSoup(response.text, "html.parser") 
    opinions = page_dom.select("div.js_product-review") 
    for opinion in opinions: 
        single_opinion = { 
            key: extract_content(opinion, *value) 
                for key, value in selectors.items() 
        }
        all_opinions.append(single_opinion)  
    try: 
        url = "https://www.ceneo.pl"+extract_content(page_dom,"a.pagination__next","href") 
    except TypeError: 
        url = None

https://www.ceneo.pl/108290707#tab=reviews
<div class="user-post user-post__card js_product-review" data-entry-id="17349010">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/6.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
KamilZ1</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2023-03-31 23:03:01">2 lata temu, </time>
<time datetime="2021-08-05 17:01:22">po 2 latach</time> użytkowania            </span>
</span>
<div class="user-p

KeyError: None

In [ ]:
if not os.path.exists("opinions"): 
    os.mkdir("opinions") 
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf: 
    json.dump(all_opinions, jf, indent=4 , ensure_ascii=False)



In [ ]:
len(all_opinions)

In [75]:
for key, value in selectors.items():
    print(key)
    print(extract_content(opinion, *value))

# single_opinion = { 
#     key: extract_content(opinion, *value) 
#         for key, value in selectors.items() 
# }

opinion_id
17349010
author


KeyError: None